# Member acquisition strategy for Cyclistic - a bike-share company

# Introduction

* **Company**: Cyclistic is a bike-share company in Chicago.

*  **Business Objective**: Membership is the main source of revenue. In order to maximize the number
of membership, the director of marketing wants to design a new marketing strategy to convert
casual riders into member riders.


# Data Analyst Task
**Identify the difference in usage behavior between the 2 groups:**

**1. Casual riders:** Riders who do not have any membership

**2. Member riders:** Riders who have membership**

This insight will be used to suggest a new marketing strategy to convert casual into member riders.

# Prepare

**Data Source:** Last 12 months (July 2020 — June 2021) of historical trip data. 
The dataset consists of twelve separate ‘.csv’ formatted files representing each month. The datasets are located in the [company’s cloud server](https://divvy-tripdata.s3.amazonaws.com/index.html).

**Tools:** **BigQuery SQL** and **Excel** for data cleaning and analysis
           
         


# Cleaning

#### Transform and inspect data with Excel

- Create backup of original files
- File naming convention: YYYYMM
- add row "day_of_week" and calculate the day of the week that each ride started using the “WEEKDAY”
command `WEEKDAY(C2,1)`. Format as General (noting that 1 = Sunday and 7 = Saturday).
- Check column format and column names
- Remove duplicates with `Remove Duplicates function`
- Inspect the data with `filter function` 
- Detect cells in station_name column contain "BIKE CHECKING", TEMP or "TESTING" which indicate test rides by the company ==> these values will be removed in BigQuery.

#### Clean and transform data with BigQuery 
- Merge 12 tables and remove unnecessary columns

`CREATE TABLE cyclistic.12_months AS 
    SELECT 
        * EXCEPT (start_station_id, end_station_id, start_lat, start_lng, end_lat, end_lng, rideable_type, member_casual),
        rideable_type AS bike_type, member_casual AS customer_type
    FROM cyclistic.202007
    UNION ALL 
    SELECT  
        * EXCEPT (start_station_id, end_station_id, start_lat, start_lng, end_lat, end_lng, rideable_type, member_casual),
        rideable_type AS bike_type, member_casual AS customer_type 
    FROM cyclistic.202008
    UNION ALL 
    SELECT  
        * EXCEPT (start_station_id, end_station_id, start_lat, start_lng, end_lat, end_lng, rideable_type, member_casual),
        rideable_type AS bike_type, member_casual AS customer_type 
    FROM cyclistic.202009
    UNION ALL 
    SELECT  
        * EXCEPT (start_station_id, end_station_id, start_lat, start_lng, end_lat, end_lng, rideable_type, member_casual),
        rideable_type AS bike_type, member_casual AS customer_type 
    FROM cyclistic.202010
    UNION ALL 
    SELECT  
        * EXCEPT (start_station_id, end_station_id, start_lat, start_lng, end_lat, end_lng, rideable_type, member_casual),
        rideable_type AS bike_type, member_casual AS customer_type 
    FROM cyclistic.202011
    UNION ALL 
    SELECT  
        * EXCEPT (start_station_id, end_station_id, start_lat, start_lng, end_lat, end_lng, rideable_type, member_casual),
        rideable_type AS bike_type, member_casual AS customer_type 
    FROM cyclistic.202012
    UNION ALL 
    SELECT  
        * EXCEPT (start_station_id, end_station_id, start_lat, start_lng, end_lat, end_lng, rideable_type, member_casual),
        rideable_type AS bike_type, member_casual AS customer_type 
    FROM cyclistic.202101
    UNION ALL 
    SELECT  
        * EXCEPT (start_station_id, end_station_id, start_lat, start_lng, end_lat, end_lng, rideable_type, member_casual),
        rideable_type AS bike_type, member_casual AS customer_type 
    FROM cyclistic.202102
    UNION ALL 
    SELECT  
        * EXCEPT (start_station_id, end_station_id, start_lat, start_lng, end_lat, end_lng, rideable_type, member_casual),
        rideable_type AS bike_type, member_casual AS customer_type 
    FROM cyclistic.202103
    UNION ALL 
    SELECT  
        * EXCEPT (start_station_id, end_station_id, start_lat, start_lng, end_lat, end_lng, rideable_type, member_casual),
        rideable_type AS bike_type, member_casual AS customer_type 
    FROM cyclistic.202104
    UNION ALL 
    SELECT  
        * EXCEPT (start_station_id, end_station_id, start_lat, start_lng, end_lat, end_lng, rideable_type, member_casual),
        rideable_type AS bike_type, member_casual AS customer_type 
    FROM cyclistic.202105
    UNION ALL 
    SELECT  
        * EXCEPT (start_station_id, end_station_id, start_lat, start_lng, end_lat, end_lng, rideable_type, member_casual),
        rideable_type AS bike_type, member_casual AS customer_type 
    FROM cyclistic.202106
    ORDER BY 
    started_at`

- Add columns "trip_duration", "ride_month" and convert day_of_week column for better understanding

`WITH
    agg_table AS(
        SELECT
            * EXCEPT (day_of_week),
            DATETIME_DIFF(ended_at, started_at,minute) AS ride_duration,
            EXTRACT(month FROM started_at) AS ride_month,
            CASE
                WHEN day_of_week = 1 THEN "Sunday"
                WHEN day_of_week = 2 THEN "Monday"
                WHEN day_of_week = 3 THEN "Tuesday"
                WHEN day_of_week = 4 THEN "Wednesday"
                WHEN day_of_week = 5 THEN "Thursday"
                WHEN day_of_week = 6 THEN "Friday"
            ELSE "Saturday"
            END AS day_of_week
        FROM 
            cyclistic.12_months`
            
- Trim string columns

`    trim_station AS (
        SELECT 
            *EXCEPT (start_station_name, end_station_name),
            TRIM (start_station_name) AS start_station_name,
            TRIM (end_station_name) AS end_station_name
        FROM agg_table 
        WHERE ride_id IS NOT NULL`

- Filter out test rides and rides last under 1 min

`remove_test_rides AS (
        SELECT 
            *
        FROM trim_station 
        WHERE 
            start_station_name NOT LIKE '%TEST%' 
        AND start_station_name NOT LIKE '%TEMP%'
        AND start_station_name NOT LIKE '%CHECKING%'
        AND start_station_name NOT LIKE '%*%'
        AND end_station_name NOT LIKE '%TEST%' 
        AND end_station_name NOT LIKE '%TEMP%'
        AND end_station_name NOT LIKE '%CHECKING%'
        AND end_station_name NOT LIKE '%*%'
        AND ride_duration >1 
    )`


=> The final table contains 3,946,100 rows from 4,460,151 rows of the original 12 months table





# Analyze

- Total number of rides by customer type 

`SELECT 
   customer_type, COUNT(customer_type) as no_of_ride
FROM 
    capstone-project-321705.cyclistic.summary_table
GROUP BY 
    customer_type`

- Average ride time by customer type

`SELECT 
   customer_type, AVG(ride_duration) AS Avg_ride_time
FROM
    capstone-project-321705.cyclistic.summary_table
GROUP BY 
    customer_type`

- Average ride time by day of week per customer type

`SELECT 
   customer_type, 
   AVG(ride_duration) AS Avg_ride_time,
   day_of_week
FROM
    capstone-project-321705.cyclistic.summary_table
GROUP BY 
    customer_type,
    day_of_week`

- Average ride time by month per customer type 

`SELECT 
   customer_type, 
   AVG(ride_duration) AS Avg_ride_time,
   ride_month
FROM
    capstone-project-321705.cyclistic.summary_table
GROUP BY 
    customer_type,
    ride_month`


- Number of rides by bike type

`SELECT 
    bike_type, COUNT (bike_type) AS no_of_ride, customer_type
FROM 
    capstone-project-321705.cyclistic.summary_table
GROUP BY 
    bike_type, customer_type`



- Number of rides by day of week per customer type

`SELECT 
   customer_type, day_of_week, COUNT(customer_type) AS No_of_rides
FROM
    capstone-project-321705.cyclistic.summary_table
GROUP BY 
    day_of_week, customer_type`

- Number of rides by month per customer type

`SELECT 
   customer_type, ride_month, COUNT(customer_type) AS No_of_rides
FROM
    capstone-project-321705.cyclistic.summary_table
GROUP BY 
    ride_month, customer_type`

- No of rides by hour of the day

`SELECT 
    EXTRACT (HOUR FROM started_at) as Hour, 
    COUNT(started_at) as num_of_trips, 
    customer_type
FROM cyclistic.summary_table
GROUP BY Hour, customer_type
ORDER BY Hour`


# Findings and visualization 

I used Tableau to visualize the data. Here are what the data tell us.

**[Link](https://drive.google.com/file/d/1DWjzKip8OsLooMxvF3Bi6Mh3wGppz3wB/view)**